# Lagrange's Equation with a constraint
## The simple pendulum
Derivation built inclass on 25 Mar 2019

In [1]:
using Pkg
Pkg.activate(".")
using SymPy

Let's define the symbols for the problem

In [2]:
m,g, l = symbols("m,g, l", positive=true)
t = symbols("t", real=true)
x = SymFunction("x")(t)
y = SymFunction("y")(t)
λ = SymFunction("lambda")(t)
;

Now we can build the (uncontrained) kinetic and potential energies, the holonomic (geometric) constraint $h(x,y)$, and the augmented Lagrangian

In [3]:
T = 1//2*m*( diff(x,t)^2 + diff(y,t)^2 )
V = m*g*y
h = x^2 + y^2 - l^2
L = T-V + λ*h

              /          2             2\                                   
              |/d       \    /d       \ |                                   
            m*||--(x(t))|  + |--(y(t))| |                                   
              \\dt      /    \dt      / /   /   2    2       2   \          
-g*m*y(t) + ----------------------------- + \- l  + x (t) + y (t)/*lambda(t)
                          2                                                 

Constructing Lagrange's equation (in the same way we did for an unconstrained system)

In [4]:
eom = [diff( L, diff(q,t), t ) - diff(L,q) for q in [x,y] ]

2-element Array{Sym,1}:
       m*Derivative(x(t), (t, 2)) - 2*lambda(t)*x(t)
 g*m + m*Derivative(y(t), (t, 2)) - 2*lambda(t)*y(t)

To solve these equations using methods we are familar with, means we need to differentiate $h$ twice, and eliminate $\lambda$.

In [5]:
eom2 = vcat(eom, [diff(h,t,t)])

3-element Array{Sym,1}:
                                                                     m*Derivative(x(t), (t, 2)) - 2*lambda(t)*x(t)
                                                               g*m + m*Derivative(y(t), (t, 2)) - 2*lambda(t)*y(t)
 2*(x(t)*Derivative(x(t), (t, 2)) + y(t)*Derivative(y(t), (t, 2)) + Derivative(x(t), t)^2 + Derivative(y(t), t)^2)

In [6]:
sol = solve(eom2, [diff(x,t,t), diff(y,t,t), λ])

Dict{Sym,Sym} with 3 entries:
  Derivative(y(t), (t, 2)) => -(g*x(t)^2 + (Derivative(x(t), t)^2 + Derivative(…
  lambda(t)                => m*(g*y(t) - Derivative(x(t), t)^2 - Derivative(y(…
  Derivative(x(t), (t, 2)) => (g*y(t) - Derivative(x(t), t)^2 - Derivative(y(t)…

## How does this compare to the usual equation?
For a simple pendulum where $\theta$ is measured counter-clockwise from the horizontal we can state that any position 
$$(x,y) = \left(l \cos\theta,\, l\sin\theta\right)$$
We can differentiate that as needed to eliminate $x$ and $y$ from the previous odes.

For reference, the equation of motion would be
$$ \ddot{\theta} + \frac{g}{l}\cos\theta = 0$$
(yes it is cosine here due to where the angle is referenced)


In [7]:
θ = SymFunction("theta")(t)
rule1= Dict( x=> l*cos(θ), y=>l*sin(θ) )
rule2 = Dict( diff(x,t) => diff(x |> subs(rule1), t), diff(y,t) => diff(y |> subs(rule1), t) )
rule3 = Dict( diff(x,t,t) => diff(x |> subs(rule1), t,t), diff(y,t,t) => diff(y |> subs(rule1), t,t) )

Dict{Sym,Sym} with 2 entries:
  Derivative(y(t), (t, 2)) => -l*(sin(theta(t))*Derivative(theta(t), t)^2 - cos…
  Derivative(x(t), (t, 2)) => -l*(sin(theta(t))*Derivative(theta(t), (t, 2)) + …

In [8]:
diff(x,t,t) - sol[diff(x,t,t)] |> subs(rule3) |>subs(rule2) |> subs(rule1) |> simplify

 /                      2          \              
 |                     d           |              
-|g*cos(theta(t)) + l*---(theta(t))|*sin(theta(t))
 |                      2          |              
 \                    dt           /              

In [9]:
diff(y,t,t) - sol[diff(y,t,t)] |> subs(rule3) |>subs(rule2) |> subs(rule1) |> simplify

/                      2          \              
|                     d           |              
|g*cos(theta(t)) + l*---(theta(t))|*cos(theta(t))
|                      2          |              
\                    dt           /              

We can see that both equations collapse down to the "regular" ode.